In [78]:
from langchain.llms import OpenAI
from api_keys import openai_apikey, huggingface_apikey
import warnings
warnings.filterwarnings(action='ignore')

In [79]:
import os
os.environ["OPEN_API_KEY"]=openai_apikey

In [80]:
llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)

In [81]:
text="What is the capital of India"

print(llm.predict(text))

?

The capital of India is New Delhi.


In [82]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=huggingface_apikey

In [83]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [84]:
output=llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

moscow


In [85]:
output=llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [86]:
llm.predict("Can you write a poem about AI")

"\n\nAI is a new technology,\nIt's a marvel of modernity,\nIt's a world of possibilities,\nIt's always ready to please.\n\nIt can learn and understand,\nIt can help us take command,\nIt can solve problems we can't,\nIt can make our lives more grand.\n\nIt can be used to predict,\nIt can help us detect,\nIt can give us insights,\nWe can use to perfect.\n\nIt can be used to automate,\nIt can help us innovate,\nIt can be used to create,\nA future that is great.\n\nAI is here to stay,\nIt's here to help us in every way,\nIt's here to make our lives easier,\nAnd to make our future brighter."

### Prompt Templates And LLMChain

In [87]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="USA")

'Tell me the capital of this USA'

In [88]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("USA"))



The capital of the United States is Washington, D.C.


### Combining Multiple Chains Uing simple Sequential Chain

In [89]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")


In [90]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [91]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("Italy")

' \n\n1. Colosseum - The iconic symbol of Rome, the Colosseum is one of the most impressive structures in the world. Built in the first century AD, it was the largest amphitheater of the Roman Empire and could hold up to 50,000 spectators.\n\n2. St. Peter’s Basilica - One of the most important churches in the world, St. Peter’s Basilica is the mother church of the Catholic faith. Located in Vatican City, the basilica is renowned for its Renaissance art and architecture.\n\n3. Pantheon - The Pantheon is a temple dedicated to all the gods of Ancient Rome. Built in the 2nd century AD, it is one of the best preserved monuments of Ancient Rome and still stands today.\n\n4. Trevi Fountain - Located in the heart of Rome, the Trevi Fountain is one of the most iconic monuments in the city. Built in the 18th century, it is a popular tourist attraction and is known for its stunning Baroque architecture.\n\n5. Sistine Chapel - Located in the Vatican City, the Sistine Chapel is famous for its Renai

### Sequential Chain

In [92]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [93]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [94]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [96]:
chain({'country':"France"})

{'country': 'France',
 'capital': '\n\nParis.',
 'places': '\n\n1. Eiffel Tower\n2. Louvre Museum\n3. Notre Dame Cathedral\n4. Arc de Triomphe\n5. Champs-Élysées\n6. Sacré-Coeur\n7. Centre Pompidou\n8. Luxembourg Gardens\n9. The Catacombs of Paris\n10. Palace of Versailles'}

### Chatmodels With ChatOpenAI

In [97]:
from langchain.chat_models import ChatOpenAI

In [98]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [99]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [100]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='Sure, here are a few AI-themed punchlines for you:\n\n1. "Why did the AI go on a diet? It wanted to shed some \'byte\'!"\n2. "I asked my AI assistant if it had a sense of humor. It replied, \'I find your jokes \'byte\'-sized amusing!\'"\n3. "My AI assistant is a great listener, but it still doesn\'t understand my jokes. It\'s like talking to a \'laugh\'-gory algorithm!"\n4. "I tried teaching my AI how to tell jokes, but it kept saying, \'I\'m sorry, I can\'t \'compute\' humor!\'"\n5. "I asked Siri if she could make me laugh. She replied, \'I\'m sorry, I\'m not programmed to \'crack\' jokes. Maybe try Google Assistant?\'"\n\nRemember, comedy is subjective, so these punchlines might not resonate with everyone. But I hope they bring a smile to your face!')

### Prompt Template + LLM +Output Parsers

In [101]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [102]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [103]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words antonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [104]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [105]:
chain.invoke({"text":"terrible"})

['great', ' wonderful', ' excellent', ' fantastic', ' amazing']